# Retrival of info by AI agents

In [1]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv("OPEN_AI_API_KEY")
)

First we have to define knowlwdge base for our retrival

In [2]:
kb = """{
    "records": [
        {
            "id": 1,
            "question": "What is the return policy?",
            "answer": "Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days."
        },
        {
            "id": 2,
            "question": "Do you ship internationally?",
            "answer": "Yes, we ship to over 50 countries worldwide. International shipping typically takes 7-14 business days and costs vary by destination. Please note that customs fees may apply."
        },
        {
            "id": 3,
            "question": "What payment methods do you accept?",
            "answer": "We accept Visa, Mastercard, American Express, PayPal, and Apple Pay. All payments are processed securely through our encrypted payment system."
        }
    ]
}"""

In [3]:
import json
def search_kb(question: str):
    """Load the whole knowledge base as json file"""
    return json.loads(kb)

STEP - 1

Describe the knowledge base tool 

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_kb",
            "description": "Get the answer to the users question from the knowledge base",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string"}
                },
                "required": ["question"],
                "additionalProperties": False,
            },

            "strict": True,
        },
    },
]

In [5]:
system_prompt = "you are helpful assistant that answers users questions about about the ecommerce store from the knowledge base"

In [6]:
messages = [{"role": "system", "content": system_prompt}, {
        "role": "user", "content": "what is your return policy"}]

In [7]:
chat_completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)

In [8]:
chat_completion.model_dump()

{'id': 'chatcmpl-B8YCYt4YUdrOct58pyhK9cQiRJgcD',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_4NzAXy5PrsRvAz3rOnF0P2Oy',
      'function': {'arguments': '{"question":"return policy"}',
       'name': 'search_kb'},
      'type': 'function'}]}}],
 'created': 1741377794,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_eb9dce56a8',
 'usage': {'completion_tokens': 17,
  'prompt_tokens': 73,
  'total_tokens': 90,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [9]:
def call_function(name, args):
    if name == "search_kb":
        return search_kb(**args)

In [10]:
for tool_call in chat_completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    messages.append(chat_completion.choices[0].message)
    
    result = call_function(name=name, args=args)

    messages.append({"role": "tool", "tool_call_id": tool_call.id, "content":json.dumps(result)})

In [11]:
from pydantic import BaseModel, Field
class KBResponse(BaseModel):
    response: str = Field(description="answer to the users question")
    source: int = Field(description="the record id of the answer")

In [12]:
chat_completion2 = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    response_format=KBResponse,
)

In [13]:
chat_completion2.model_dump()

{'id': 'chatcmpl-B8YCZzjND6N4gtDYjCky43cWU1tdC',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{"response":"Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days.","source":1}',
    'refusal': None,
    'role': 'assistant',
    'audio': None,
    'function_call': None,
    'tool_calls': None,
    'parsed': {'response': 'Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days.',
     'source': 1}}}],
 'created': 1741377795,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_eb9dce56a8',
 'usage': {'completion_tokens': 45,
  'prompt_tokens': 319,
  'total_tokens': 364,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens

In [17]:
final_response = chat_completion2.choices[0].message.parsed

In [20]:
final_response.response

'Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days.'

In [21]:
final_response.source

1